In [4]:
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512,84),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(84,10)
            
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


def VGG11():
    return VGG('VGG11')


def VGG13():
    return VGG('VGG13')


def VGG16():
    return VGG('VGG16')


def VGG19():
    return VGG('VGG19')

In [14]:
import torch.optim as optim
import torch.utils.data
from torch.utils.data import sampler
import torch.backends.cudnn as audnn
import torchvision
from torchvision import transforms
import numpy as np
import time
from tensorboardX import SummaryWriter
import torchvision.models as models
from random import randint
start=time.time()

writer=SummaryWriter()

lr=1e-3
epoch=300
trainBatchSize=256
testBatchSize=100
NUM_TRAIN = 49000

class Solver():
    def __init__(self):
        self.model=None
        self.lr=lr
        self.epochs=epoch
        self.train_batch_size=trainBatchSize
        self.test_batch_size=testBatchSize
        self.optimizer=None
        self.device=None
        self.cuda=torch.cuda.is_available()
        self.train_loader=None
        self.val_loader=None
        self.test_loader=None
        self.scheduler=None
        self.criterion =None
        self.loss=10000
        
    def load_data(self):
        
        train_transform=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        train_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=train_transform)
        self.train_loader=torch.utils.data.DataLoader(dataset=train_set,batch_size=self.train_batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
        val_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=train_transform)
        self.val_loader=torch.utils.data.DataLoader(dataset=train_set,batch_size=self.train_batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))
        test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform)
        self.test_loader=torch.utils.data.DataLoader(dataset=test_set,batch_size=self.test_batch_size)
        
    def load_model(self):
        if self.cuda:
            self.device=torch.device('cuda')#missed
            print('yes')
        else:
            slf.device=torch.device('cpu')
        
        tmp_model=models.vgg16(pretrained=True)
        #fc_features=tmp_model.fc.in_features
        #tmp_model.fc=nn.Linear(fc_features,10)
        tmp_model.classifier= nn.Sequential(
            nn.Linear(512,84),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(84,10)
            
        )
        self.model=tmp_model.to(self.device)
        #self.model=VGG16().to(self.device)
        
        #reload=torch.load('save/prmodelpara.pth')
        #self.model.load_state_dict(reload['net'])
        #for param in self.model.parameters():
            #param.requires_grad=False
        #for param in self.model.classifier.parameters():
            #param.requires_grad=True
        #ignored_params=list(map(id,self.model.classifier.parameters()))
        #base_params=filter(lambda p : id(p) not in ignored_params, self.model.parameters()) 
        #self.optimizer=optim.Adam([{'params':base_params, 'lr':0.0003},{'params':self.model.classifier.parameters(),'lr':0.001}] )
        self.optimizer=optim.Adam(filter(lambda p: p.requires_grad,self.model.parameters()),lr=self.lr)
        #self.scheduler=optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', factor=0.5, patience=2, verbose=False, threshold=0.000001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
        self.scheduler=optim.lr_scheduler.MultiStepLR(self.optimizer,milestones=[40,60,80,100,150,200,250,275],gamma=0.5)
        
        self.criterion =nn.CrossEntropyLoss().to(self.device)
        
    def train(self):
        self.model.train()
        train_loss=0
        train_correct=0
        total=0
        
        for batch_num,(data,target) in enumerate(self.train_loader):
            data,target=data.to(self.device),target.to(self.device)
            self.optimizer.zero_grad()
            output=self.model(data)
            loss=self.criterion(output,target)
            loss.backward()
            self.optimizer.step()
            train_loss+=loss.item()
            prediction=torch.max(output,1)
            total+=target.size(0)
            train_correct += np.sum(prediction[1].cpu().numpy() == target.cpu().numpy())
            #print('loss:',loss.item(),'accuracy:',train_correct/total)
            
        return train_loss , train_correct/total
    
    def test(self):
        print('test====================')
        self.model.eval()
        test_loss=0
        test_correct=0
        total=0
        
        with torch.no_grad():
            for batch_num, (data, target) in enumerate(self.val_loader):
                data, target = data.to(self.device), target.to(self.device)
                output=self.model(data)
                loss=self.criterion(output,target)
                test_loss+=loss.item()
                prediction=torch.max(output,1)
                total+=target.size(0)
                test_correct += np.sum(prediction[1].cpu().numpy() == target.cpu().numpy())
                #print ('test accuracy:',test_correct/total)
        return test_loss,test_correct/total
    
    def run(self,writer):
        self.load_data()
        self.load_model()
        accuracy=0
        pre_loss=10
        for epoch in range(1, self.epochs + 1):
            
            print("\n=====> epoch: %d/200" % epoch)
            
            train_loss,train_result=self.train()
            self.scheduler.step(epoch)
            print('epoch:',epoch)
            print ('train accuracy:',train_result,'loss:',train_loss)
            
            test_result=self.test()
            accuracy=max(accuracy,test_result[1])
            print('test accuracy:',test_result[1])
            writer.add_scalar('loss_value',train_loss,epoch)
            writer.add_scalar('train_acc',train_result,epoch)
            writer.add_scalar('test_acc',test_result[1],epoch)
            train_transform=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.Resize(randint(32,48)),transforms.CenterCrop(32),transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
            train_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=train_transform)
            self.train_loader=torch.utils.data.DataLoader(dataset=train_set,batch_size=self.train_batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
        print("===> BEST ACC. PERFORMANCE: %.3f%%" % (accuracy * 100))
            
        
        
        
    def final_test(self):
        self.model.eval()
        for x, y in  self.test_loader:
            x=x.to(self.device)
            y=y.to(self.device)
            scores=self.model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('FINNAL ACCURACY:',acc)
            
            


In [15]:

solver=Solver()
solver.run(writer)  
writer.close()
end=time.time()
print (end-start,"Final time")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
yes

=====> epoch: 1/200
epoch: 1
train accuracy: 0.2313265306122449 loss: 370.1261215209961
test====================
test accuracy: 0.374
Files already downloaded and verified

=====> epoch: 2/200
epoch: 2
train accuracy: 0.5473877551020409 loss: 229.35316401720047
test====================
test accuracy: 0.684
Files already downloaded and verified

=====> epoch: 3/200
epoch: 3
train accuracy: 0.6827551020408164 loss: 183.3664971590042
test====================
test accuracy: 0.731
Files already downloaded and verified

=====> epoch: 4/200
epoch: 4
train accuracy: 0.7691632653061224 loss: 140.12366449832916
test====================
test accuracy: 0.74
Files already downloaded and verified

=====> epoch: 5/200
epoch: 5
train accuracy: 0.8025714285714286 loss: 121.2653024494648
test====================
test accuracy: 0.776
Files already downloaded and verified

=====> epoch: 6

test accuracy: 0.804
Files already downloaded and verified

=====> epoch: 48/200
epoch: 48
train accuracy: 0.9816326530612245 loss: 12.150119273923337
test====================
test accuracy: 0.819
Files already downloaded and verified

=====> epoch: 49/200
epoch: 49
train accuracy: 0.9843061224489796 loss: 10.897302971221507
test====================
test accuracy: 0.815
Files already downloaded and verified

=====> epoch: 50/200
epoch: 50
train accuracy: 0.958 loss: 26.639219716191292
test====================
test accuracy: 0.878
Files already downloaded and verified

=====> epoch: 51/200
epoch: 51
train accuracy: 0.9865306122448979 loss: 9.089573858305812
test====================
test accuracy: 0.878
Files already downloaded and verified

=====> epoch: 52/200
epoch: 52
train accuracy: 0.9883673469387755 loss: 7.597132636234164
test====================
test accuracy: 0.87
Files already downloaded and verified

=====> epoch: 53/200
epoch: 53
train accuracy: 0.9606326530612245 loss: 25.9

test accuracy: 0.872
Files already downloaded and verified

=====> epoch: 95/200
epoch: 95
train accuracy: 0.9996122448979592 loss: 0.3363013965426944
test====================
test accuracy: 0.888
Files already downloaded and verified

=====> epoch: 96/200
epoch: 96
train accuracy: 0.9989183673469387 loss: 0.6202940291404957
test====================
test accuracy: 0.861
Files already downloaded and verified

=====> epoch: 97/200
epoch: 97
train accuracy: 0.9983673469387755 loss: 1.0340208629495464
test====================
test accuracy: 0.873
Files already downloaded and verified

=====> epoch: 98/200
epoch: 98
train accuracy: 0.999734693877551 loss: 0.24285502900602296
test====================
test accuracy: 0.878
Files already downloaded and verified

=====> epoch: 99/200
epoch: 99
train accuracy: 0.9897959183673469 loss: 7.1117617366835475
test====================
test accuracy: 0.899
Files already downloaded and verified

=====> epoch: 100/200
epoch: 100
train accuracy: 0.998673469

Files already downloaded and verified

=====> epoch: 141/200
epoch: 141
train accuracy: 0.999938775510204 loss: 0.07399833789804688
test====================
test accuracy: 0.891
Files already downloaded and verified

=====> epoch: 142/200
epoch: 142
train accuracy: 0.997265306122449 loss: 2.018023297583568
test====================
test accuracy: 0.902
Files already downloaded and verified

=====> epoch: 143/200
epoch: 143
train accuracy: 0.9998367346938776 loss: 0.16582621798784203
test====================
test accuracy: 0.888
Files already downloaded and verified

=====> epoch: 144/200
epoch: 144
train accuracy: 0.9995102040816326 loss: 0.2679161856023029
test====================
test accuracy: 0.89
Files already downloaded and verified

=====> epoch: 145/200
epoch: 145
train accuracy: 0.999734693877551 loss: 0.22604518039952382
test====================
test accuracy: 0.882
Files already downloaded and verified

=====> epoch: 146/200
epoch: 146
train accuracy: 0.9979591836734694 loss:


=====> epoch: 187/200
epoch: 187
train accuracy: 0.9999591836734694 loss: 0.036475605689702206
test====================
test accuracy: 0.898
Files already downloaded and verified

=====> epoch: 188/200
epoch: 188
train accuracy: 0.9994081632653061 loss: 0.43866585998330265
test====================
test accuracy: 0.892
Files already downloaded and verified

=====> epoch: 189/200
epoch: 189
train accuracy: 0.9997142857142857 loss: 0.23857473403040785
test====================
test accuracy: 0.892
Files already downloaded and verified

=====> epoch: 190/200
epoch: 190
train accuracy: 0.9998979591836735 loss: 0.07597133990111615
test====================
test accuracy: 0.896
Files already downloaded and verified

=====> epoch: 191/200
epoch: 191
train accuracy: 0.9999795918367347 loss: 0.04222525921250053
test====================
test accuracy: 0.896
Files already downloaded and verified

=====> epoch: 192/200
epoch: 192
train accuracy: 0.999734693877551 loss: 0.19880759750958532
test======

epoch: 233
train accuracy: 0.9998979591836735 loss: 0.10595455224840755
test====================
test accuracy: 0.901
Files already downloaded and verified

=====> epoch: 234/200
epoch: 234
train accuracy: 0.9999591836734694 loss: 0.027189653438654204
test====================
test accuracy: 0.905
Files already downloaded and verified

=====> epoch: 235/200
epoch: 235
train accuracy: 1.0 loss: 0.011597290849749697
test====================
test accuracy: 0.895
Files already downloaded and verified

=====> epoch: 236/200
epoch: 236
train accuracy: 0.9995918367346939 loss: 0.32294373674085364
test====================
test accuracy: 0.877
Files already downloaded and verified

=====> epoch: 237/200
epoch: 237
train accuracy: 0.9998979591836735 loss: 0.04714243142734631
test====================
test accuracy: 0.885
Files already downloaded and verified

=====> epoch: 238/200
epoch: 238
train accuracy: 0.9999795918367347 loss: 0.012828384382793878
test====================
test accuracy: 0.892

test accuracy: 0.902
Files already downloaded and verified

=====> epoch: 280/200
epoch: 280
train accuracy: 0.999938775510204 loss: 0.03254428806786791
test====================
test accuracy: 0.904
Files already downloaded and verified

=====> epoch: 281/200
epoch: 281
train accuracy: 0.9999795918367347 loss: 0.013835236071969348
test====================
test accuracy: 0.903
Files already downloaded and verified

=====> epoch: 282/200
epoch: 282
train accuracy: 0.9999183673469387 loss: 0.08442452922463417
test====================
test accuracy: 0.906
Files already downloaded and verified

=====> epoch: 283/200
epoch: 283
train accuracy: 1.0 loss: 0.005139883750871377
test====================
test accuracy: 0.905
Files already downloaded and verified

=====> epoch: 284/200
epoch: 284
train accuracy: 0.9999795918367347 loss: 0.043309994113542416
test====================
test accuracy: 0.9
Files already downloaded and verified

=====> epoch: 285/200
epoch: 285
train accuracy: 1.0 loss: 0

In [12]:
solver.model.eval()
test_transform=transforms.Compose( [transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
num_correct=0
num_samples=0
test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform) 
test_loader=torch.utils.data.DataLoader(dataset=test_set,batch_size=100)
for x, y in  test_loader:
                x=x.to(solver.device)
                y=y.to(solver.device)
                scores=solver.model(x)
                _, preds = scores.max(1)
                num_correct += (preds == y).sum()
                num_samples += preds.size(0)
                acc = float(num_correct) / num_samples
x=x.to(torch.device("cpu"))
    
print('FINNAL ACCURACY:',acc)

Files already downloaded and verified
FINNAL ACCURACY: 0.7279


In [5]:
#WATCHOUT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#WATCHOUT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#watchout!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#WATCHOUT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
state={'net':solver.model.state_dict(),'optimizer':solver.optimizer.state_dict(),
      'epoch':epoch}
torch.save(state,'save/prmodelpara.pth')


In [3]:
lr=1e-3
#reload=torch.load('save/modelpara.pth')
solver=Solver()
solver.load_model()
#solver.model.load_state_dict(reload['net'])
#solver.optimizer.load_state_dict(reload['optimizer'])
solver.run(writer)  
writer.close()


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

=====> epoch: 1/200
train accuracy: 0.46814285714285714 loss: 689.1059673428535
test====================
test accuracy: 0.603

=====> epoch: 2/200
train accuracy: 0.6791836734693878 loss: 442.1382910013199
test====================
test accuracy: 0.714

=====> epoch: 3/200
train accuracy: 0.7572448979591837 loss: 344.2378862798214
test====================
test accuracy: 0.761

=====> epoch: 4/200
train accuracy: 0.8013673469387755 loss: 286.52477300167084
test====================
test accuracy: 0.786

=====> epoch: 5/200
train accuracy: 0.8291632653061225 loss: 248.1155005991459
test====================
test accuracy: 0.798

=====> epoch: 6/200
train accuracy: 0.8520612244897959 loss: 214.62967759370804
test====================
test accuracy: 0.84

=====> epoch: 7/200
train accuracy: 0.8706122448979592 loss: 188.4897862225771
test====================
test accuracy: 0.85

==

test accuracy: 0.895

=====> epoch: 65/200
train accuracy: 0.9948571428571429 loss: 7.2782187898192205
test====================
test accuracy: 0.875

=====> epoch: 66/200
train accuracy: 0.9927959183673469 loss: 10.137019643836538
test====================
test accuracy: 0.883

=====> epoch: 67/200
train accuracy: 0.9946326530612245 loss: 8.023708734530373
test====================
test accuracy: 0.896

=====> epoch: 68/200
train accuracy: 0.9932244897959184 loss: 9.351669086856418
test====================
test accuracy: 0.893

=====> epoch: 69/200
train accuracy: 0.9945102040816326 loss: 7.68913628111477
test====================
test accuracy: 0.904

=====> epoch: 70/200
train accuracy: 0.9951428571428571 loss: 7.2610335960562224
test====================
test accuracy: 0.893

=====> epoch: 71/200
train accuracy: 0.9944489795918368 loss: 7.7091634613025235
test====================
test accuracy: 0.89

=====> epoch: 72/200
train accuracy: 0.994734693877551 loss: 8.09813584764197
test=====

In [11]:

reload=torch.load('save/modelpara1.pth')
model=VGG16().to(torch.device('cuda'))
model.load_state_dict(reload['net'])
model.eval()
test_transform=transforms.Compose( [transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
num_correct=0
num_samples=0
test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform) 
test_loader=torch.utils.data.DataLoader(dataset=test_set,batch_size=100)
for x, y in  test_loader:
                x=x.to(solver.device)
                y=y.to(solver.device)
                scores=model(x)
                _, preds = scores.max(1)
                num_correct += (preds == y).sum()
                num_samples += preds.size(0)
                acc = float(num_correct) / num_samples

    
print('FINNAL ACCURACY:',acc)

Files already downloaded and verified
FINNAL ACCURACY: 0.9057
